NotFoundError: Could not find directory ships32

In [11]:
%cd /workspace

/workspace


In [12]:
import os
print("cwd:", os.getcwd())
print("dirs:", os.listdir())


cwd: /workspace
dirs: ['rayan.py', '.config', '.bashrc', 'ships.tgz', 'rayan.ipynb', '.bash_profile', '.local', 'ships_competition.npz', 'ships32', '.bash_history', 'amine', '.keras', '.cache', 'wget-log', '.conda', '.workspace.initialized', '.ipython', 'wget-log.1', 'ai-training-examples', '.nv', '.ssh', '.miniconda3']


In [13]:
import tensorflow as tf, numpy as np, pandas as pd
from tensorflow.keras import layers, models, callbacks, mixed_precision

# ---------- data ----------
train_ds = tf.keras.utils.image_dataset_from_directory(
    "ships32", validation_split=0.15, subset="training",
    seed=42, label_mode="int", image_size=(32, 32), batch_size=256)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "ships32", validation_split=0.15, subset="validation",
    seed=42, label_mode="int", image_size=(32, 32), batch_size=256)

aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (aug(x) / 255., y), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds   = val_ds.map(lambda x, y: (x / 255., y),     num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

Found 48330 files belonging to 13 classes.
Using 41081 files for training.
Found 48330 files belonging to 13 classes.
Using 7249 files for validation.


In [14]:


# ---------- model (<30 layers) ----------
mixed_precision.set_global_policy('mixed_float16')

def block(x, f):
    x = layers.Conv2D(f, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)              # ← changement ici
    return x

inp = layers.Input((32, 32, 3))
x = block(inp, 32);  x = block(x, 32);  x = layers.MaxPool2D()(x)
x = block(x, 64);   x = block(x, 64);   x = layers.MaxPool2D()(x)
x = block(x, 128);  x = block(x, 128);  x = layers.MaxPool2D()(x)
x = block(x, 256)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
out = layers.Dense(13, activation='softmax', dtype='float32')(x)

model = models.Model(inp, out)
print("Nombre de couches :", len(model.layers))  # 28

model.compile(
    optimizer=tf.keras.optimizers.Adam(3e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cbs = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(factor=0.5, patience=4, verbose=1)
]


model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=cbs)




Nombre de couches : 28
Epoch 1/100


I0000 00:00:1745019774.793002   53776 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


161/161 [==============================] - 13s 25ms/step - loss: 1.8421 - accuracy: 0.3900 - val_loss: 3.3628 - val_accuracy: 0.1248 - lr: 0.0030
Epoch 2/100
161/161 [==============================] - 3s 15ms/step - loss: 1.3266 - accuracy: 0.5512 - val_loss: 4.0510 - val_accuracy: 0.3526 - lr: 0.0030
Epoch 3/100
161/161 [==============================] - 2s 14ms/step - loss: 1.1427 - accuracy: 0.6119 - val_loss: 1.4319 - val_accuracy: 0.5624 - lr: 0.0030
Epoch 4/100
161/161 [==============================] - 2s 14ms/step - loss: 1.0472 - accuracy: 0.6432 - val_loss: 1.3608 - val_accuracy: 0.5398 - lr: 0.0030
Epoch 5/100
161/161 [==============================] - 2s 14ms/step - loss: 0.9810 - accuracy: 0.6638 - val_loss: 1.4103 - val_accuracy: 0.5751 - lr: 0.0030
Epoch 6/100
161/161 [==============================] - 2s 14ms/step - loss: 0.9247 - accuracy: 0.6808 - val_loss: 1.1698 - val_accuracy: 0.6148 - lr: 0.0030
Epoch 7/100
161/161 [==============================] - 2s 14ms/step -

In [15]:
# ---------- submission ----------
comp = np.load('ships_competition.npz')['X'] / 255.
pred = model.predict(comp, batch_size=512).argmax(1)
pd.DataFrame({'ID': np.arange(len(pred)), 'Category': pred}).to_csv('ships_competition.csv', index=False)

4/4 [==============================] - 1s 100ms/step
